# AI4ER Environmental Risk Practical Session: Image Analysis of Volcanic Rocks
- Friday 6th November, 2020 - online session.
- Lecturer: John Maclennan
- Practical Experts: Matt Ball and Norbert Toth.

## Purpose of this session
### Broader Context
Image analysis is a field with clear targets for machine learning approaches. Furthermore, many environmental applications use spectral imaging - the acquisition of a spectrum at each pixel. This is a language that will likely become familiar to you from the treatment of satellite data to map features on the global scale. At the other extreme of spatial scales, the examination of environmental materials by electron microscopy is evolving rapidly due to rapid advances in instrumentation the application of techniques developed in materials science to earth and environmental problems. 
### Key Skills
- Run Jupyter Notebook in Jupyter Lab from Binder
- Use of HyperSpy package for handling electron microscope data
- Use of Skimage package for image processing
- Application of Machine Learning techniques to Phase Identification
- Masking and Segmentation of phases
- Extraction of rock textural properties (Crystal Sizes and Shapes)
- Estimating magmatic timescales

## Table of Contents <a id='link1'></a>

<a href='#link10'> **1.0 What is a Phase Map of a Volcanic Rock?**</a>

<a href='#link10'> **2.0 Using Machine Learning to guide phase mapping**</a>

<a href='#link2'> **1.0 Loading and Saving data**</a>

<a href='#link3'> **2.0 Machine Learning for Separating Phases**</a>

<a href='#link6'> **2.1 Identifying the number of phases**</a>

<a href='#link7'> **2.2 Creating a phase mask**</a>

<a href='#link4'> **3.0 Quantification of Phases - Mineralogy**</a>

<a href='#link5'> **4.0 Quantification of Elements - Morphology**</a>

This notebook contains a commented workflow on running machine learning on EDS data for phase separation and quantifying morphology.

Original written by Matt Ball, updated by John Maclennan and with new code from Norbert Toth.

## 1.0 What is a Phase Map of a Volcanic Rock? <a id='link10'></a>

Here is an example of a [phase map](https://academic.oup.com/view-large/figure/82008133/egu058f10p.png) of a rock from the active volcanic zones of Iceland.
The image is from [Neave et al., 2014](https://academic.oup.com/petrology/article/55/12/2311/1558945).
It was generated using a commerical system called QEMSCAN that is a piece of software that is linking to a scanning electron microcope (SEM).
You can see from the scale that we are looking at map of a 'thin section' prepared from the rock - you saw some of these in the lecture slides, as well as the SEM in the Department of Earth Sciences. The rocks are made of 'phases'. Each phase has a distinctive composition and set of thermodynamic properties which are determined by the crystalline (or glass) structure of the cooled lava. These phase names (e.g. olivine, plagioclase, clinopyroxene) are displayed on the legend of the figure along with the colours. In essence the QEMSCAN system is performing an advanced form of colour-by-numbers. The question is - how does it do this?

This raw data that goes into generation of these maps is acquired using a technique known as Energy Dispersive Spectroscopy (EDS). The SEM fires an electron beam at a focussed point on the sample surface, causing electrons to shift around energy levels and then to release X-rays - which are gathered by the EDS detectors. A spectrum is generated and the position of peaks can be attributed to characteristic energies that are associated with elements. The peak height is linked to the concentration of the element in the sample.

The QEMSCAN software converts the spectrum for each pixel into an estimate of the chemical composition of that point, and then uses a series of logical filters on the compositional data to match each pixel to a phase. For example, olivine is a mineral that contains This means that a large database of phases has to be accessed and the selection of possible phases is not guided by the compositional variations observed within the sample itself.

## 2.0 Using Machine Learning to guide phase mapping: Setup <a id='link20'></a>

An alternative approach with considerable promise in this field is to base the identification of phases directly on the available EDS data. In order to understand how we might be able to use this approach we are going to work through a set of activties in this notebook, starting from data input/output through to quantification of the rock textures.

The datasets are large and some of the ML processing is computationally expensive. For this practical we will therefore work with relatively small portions of the images, but will import results from larger images that have been calculated beforehand - Matt or Norbert might well say - "Here's one that I made earlier".

### 2.1 Load in Packages - Numpy, Scipy, Skimage, Hyperspy

In [1]:
import hyperspy.api as hs
hs.preferences.GUIs.enable_traitsui_gui = False
hs.preferences.save()
import numpy as np
from numpy import mean, std, median, sqrt, exp, log, delete

import matplotlib.pyplot as plt
import matplotlib as mpl
from cycler import cycler

import scipy
from scipy import ndimage as ndi
from scipy.stats import sem, t, norm, gaussian_kde, mstats, iqr
from scipy.optimize import curve_fit

import skimage as ski
from skimage.segmentation import random_walker
from skimage.feature import peak_local_max
from skimage import exposure
from skimage import measure
from skimage import morphology as mph
from skimage import restoration
from skimage.filters import threshold_otsu
from skimage.color import rgb2gray

%matplotlib widget

/Users/johnmaclennan/opt/anaconda3/lib/python3.8/site-packages/pyUSID/viz/__init__.py:16: FutureWarning: Please use sidpy.viz.plot_utils instead of pyUSID.viz.plot_utils. pyUSID.plot_utils will be removed in a future release of pyUSID
  warn('Please use sidpy.viz.plot_utils instead of pyUSID.viz.plot_utils. '


### 2.2 Load and Plot EDS Data

Use Hyperspy to load in the EDS data in its bespoke .bcf format as output by the software that runs the detectors. You can look in the Hyperspy documentation to figure out what some of these commands are doing. 

In [2]:
#a = hs.load('your_filename.bcf')
a = hs.load('JM1_OL7_map1.bcf')
#https://drive.google.com/file/d/1yHQXNsE8Pls8lfTuLQTI9R46uqpY-Q0s/view?usp=sharing 
#gdrive_file_id = '1yHQXNsE8Pls8lfTuLQTI9R46uqpY-Q0s'
#a = hs.load(f'https://docs.google.com/uc?id={gdrive_file_id}&export=download')

a

[<Signal2D, title: Ch 0, dimensions: (|1820, 1295)>,
 <Signal2D, title: Ch 0, dimensions: (|2000, 2000)>,
 <EDSSEMSpectrum, title: EDX, dimensions: (1820, 1295|2048)>]

In [3]:
s = a[2]
s.axes_manager.set_signal_dimension(1)

In [4]:
hs.preferences.gui()

In [5]:
s.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Two new plot windows should have appeared. They does not look very impressive. One plot is a map and you can see the scale bar.
The bottom plot shows a spectrum, with a very small number of points. Can you figure out from the [Hyperspy manual](http://hyperspy.org/hyperspy-doc/current/user_guide/visualisation.html#multidimensional-spectral-data) what you have plotted here?

If you zoom in to the origin (look carefully at all the information on the plots) then you should be able to see a red line round the selected pixel. You can move this and examine the spectra of other individual pixels.

In [5]:
plt.close('all')

Try to play around with the Region of Interest commands in Hyperspy plotting. This should start to give you an even clearer feeling for what the low-quality spectrum - low number of counts - available at each point looks like.

In [6]:
roi = hs.roi.RectangularROI(left=120, right=460., top=300, bottom=560)
s.plot()
sc = roi.interactive(s)
sc.plot()

using interactive roi


In [7]:
plt.close('all')

removing widget


### 2.3 Compositional information in Spectra

Hyperspy contains a database of [elemental properties](http://hyperspy.org/hyperspy-doc/current/user_guide/eds.html#elemental-database). Add a code cell in below to print where you would expect to find peaks for Mg and Fe in the spectra. 

It is also possible to add labels to peaks found on the plotted spectra, as shown below. You might like to compare these with a high quality spectrum (lots of counts) for the mineral phase called olivine (e.g. in the [Mindat database](https://www.mindatnh.org/Forsterite%20Gallery.html)).

In [8]:
s.add_elements(['Si','Mg','Fe','Al','Ca'])
s.plot(True)

In [9]:
plt.close('all')

### 2.4 Subset the loaded data

The data file provided is very large and too big for us to process in the Binder environment (2 Gb memory). It is therefore necessary to subset the data.

In [10]:
s_mini = s.data[0:200,500:800,:]
s_min = hs.signals.Signal1D(s_mini)
s_min.plot()

In [11]:
plt.close('all')

### Assign metadata

In [12]:
s_min.set_signal_type("EDS_SEM")
s_min.axes_manager[-1].name = 'E'
s_min.axes_manager['E'].units = 'keV'
s_min.axes_manager['E'].scale = 0.01
#Set the offset, the difference between the measured peak location and the true location, often an
#artificial zero peak is added to EDS data which can be used for this. This may need to be done a few
#times to correctly set this value
s_min.axes_manager['E'].offset = -0.3
#Set the beam energy to the beam energy the EDS data was collected at in keV
s_min.metadata.Acquisition_instrument.SEM.beam_energy = 20
s_min.plot()

In [13]:
plt.close('all')

### Saving and Loading in Hyperspy Format

When files are saved in Hyperspy format they can be rapidly reloaded as required. 



In [14]:
s.save('your_filename')

Overwrite 'your_filename.hspy' (y/n)?
 y


In [15]:
s = hs.load('your_filename.hspy')

## 3.0 Machine Learning for Separating Phases <a id='link3'></a>

### 3.1 Identifying the number of phases <a id='link6'></a>

First we run a PCA (Principal Component Analysis) on the data to identify the number of significant principal components. The different phases will consist of both a loading map and corresponding spectrum. These can be cycled through using the arrow keys. Also plotted are two scree plots, which show the percentage of the data which can be explained by each principal component, one of which is plotted with a logarithmic y-axis, due to the large amount of data, this will be too slow to perform on the full dataset, so we will instead use the subset of data we created earlier, whilst the results of the machine learning on the full set can be loaded in and viewed.
The cell below produces a number of plots in windows - make sure that you can see them all and interact with them - using the slider window in particular.

In [16]:
s_min.change_dtype('float32')
s_min.decomposition(True, algorithm='SVD', output_dimension=20)
s_min.plot_decomposition_results()
s_min.plot_explained_variance_ratio(log=False)
s_min.plot_explained_variance_ratio(log=True)

Decomposition info:
  normalize_poissonian_noise=True
  algorithm=SVD
  output_dimension=20
  centre=None


<AxesSubplot:title={'center':'\nPCA Scree Plot'}, xlabel='Principal component index', ylabel='Proportion of variance'>

It is worth noting that the results of this PCA contain both negative loading intensities as well as negative spectra, neither of which is physically meaningful. This first step is really to identify the number of significant phases which the machine learning can pull apart. How many phases do you think are being picked up by this PCA approach? How can we tell whether the recovered principal components are likely to correspond to natural features of the rock sample?

In [17]:
plt.close('all')

Again we can save the results of this analysis

In [18]:
s.learning_results.save('your_filename_PCA')

Here's one I made earlier! Now we can load in the PCA results on the whole dataset to compare to our subsection. Alternatively we can load in results of any previous PCA

In [19]:
s.learning_results.load('AI4ER_JM1_OL7_PCA.npz')

In [20]:
s.plot_decomposition_results()
s.plot_explained_variance_ratio(log=False)
s.plot_explained_variance_ratio(log=True)

<AxesSubplot:title={'center':'EDX\nPCA Scree Plot'}, xlabel='Principal component index', ylabel='Proportion of variance'>

Again, how many of the PCs do you think might be picking up aspects of the rock sample - rather than analytical noise or artifacts?

In [21]:
plt.close('all')

### 3.2 Defining the number of principal components

The number of principal components can be defined either by the number which is below a critical value of variance, by the shape of the scree plot, by the form of the associated spectrum or by the spatial pattern defined in the images. Pick a number - perhaps discuss the best strategy with a demonstrator if you are unsure.

In [22]:
PC = 'number of principal components' # Replace strong with number
PC = 3

We now use this number of principal components to run a non-negative matrix factorisation, a clustering algorithm which produces only positive loadings and spectra, removing the issue of non physically meaningful results. This makes use of machine learning algorithms through scikit-learn. Again we will perform this only on the subset of the data and load in a previously collected result for the full dataset.

In [23]:
s_min.decomposition(True, algorithm='NMF', output_dimension=PC)
s_min.plot_decomposition_results()

/Users/johnmaclennan/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


Decomposition info:
  normalize_poissonian_noise=True
  algorithm=NMF
  output_dimension=3
  centre=None
scikit-learn estimator:
NMF(n_components=3)


This should be starting to look quite good - use the slider wondow to examine the maps and spectra associated with the factors obtained.  Hopefully you can see crystal shapes on the maps and note that the spectra look physically plausible - in that at least they all have positive count values. Another remarkable feature is how smooth and sharp the spectra associated with each factor are - certainly in comparison to the noisy low-count spectra from each individual point. This is the power of looked for correlated signals across thousands of pixels. 

In [24]:
plt.close('all')

Again we can save the results of this analysis

In [25]:
s.learning_results.save('your_filename_NMF')

And again here's another pre-NMF-ed set of results. We could alternatively load in other previously collected results

In [26]:
s.learning_results.load('AI4ER_JM1_OL7_NMF.npz')

Now, plot these results up and attempt to label the spectra for the position of the elements.

In [27]:
s.plot_decomposition_results()

In the next plot, you can move between the spectra of the factors by clicking on the red line in the slider and moving it vertically. 

In [30]:
snspec = s.get_decomposition_factors()
snspec.add_elements(['Si','Mg','Fe','Al','Ca'])
snspec.plot(True)

Which phases do the different factors correspond to? Use the [Mindat](https://www.mindat.org/) database to check on mineral compositions.
In this type of rock, which is called a basalt, the common mineral phases are olivine, clinopyroxene, plagioclase, magnetite and a Cr-bearing spinel.
The basaltic liquid also quenches to a glass when it cools rapidly on eruption. 
You can find some likely glass compositions (as well as the answers to the questions above) in [Larsen and Pedersen, 2000](https://academic.oup.com/petrology/article/41/7/1071/1457495#99082445) - glass compositions in Table 3.